## WSJ article on indebtedness among Masters Degree recipients at elite schools

This workbook uses the DOE dataset to investigate claims from a WSJ article around indebtedness levels for masters degree recipients at elite schools. The article singles out film, art, and certain health programs as areas where students are going deeply into debt for degrees that don't pay very well.

"Financially Hobbled for Life’: The Elite Master’s Degrees That Don’t Pay Off"

https://www.wsj.com/articles/financially-hobbled-for-life-the-elite-masters-degrees-that-dont-pay-off-11625752773

I recreate some of the calculations and queries here. No surprises, I'm getting similar numbers to the WSJ article.  

Note that I am using the same dataset refernced in the article, so I'm really veryfing calculations and queries, not the dataset itself. It would be helpful to have an independent survey for additionaal support.

In [1]:
import pandas as pd
import numpy as np
from pandasql import sqldf
pysqldf = lambda q: sqldf(q, globals())

In [2]:
df = pd.read_csv('data/FieldOfStudyData1516_1617_PP.csv')

Taking a look at the headers with data

In [3]:
pd.set_option('display.max_columns', None) 
pd.set_option('display.max_rows', None) 
pd.set_option('display.max_colwidth', None)

In [4]:
df.head(5)

,UNITID,OPEID6,INSTNM,CONTROL,MAIN,CIPCODE,CIPDESC,CREDLEV,CREDDESC,IPEDSCOUNT1,IPEDSCOUNT2,DEBT_ALL_STGP_ANY_N,DEBT_ALL_STGP_ANY_MEAN,DEBT_ALL_STGP_ANY_MDN,DEBT_ALL_STGP_EVAL_N,DEBT_ALL_STGP_EVAL_MEAN,DEBT_ALL_STGP_EVAL_MDN,DEBT_ALL_PP_ANY_N,DEBT_ALL_PP_ANY_MEAN,DEBT_ALL_PP_ANY_MDN,DEBT_ALL_PP_EVAL_N,DEBT_ALL_PP_EVAL_MEAN,DEBT_ALL_PP_EVAL_MDN,DEBT_MALE_STGP_ANY_N,DEBT_MALE_STGP_ANY_MEAN,DEBT_MALE_STGP_ANY_MDN,DEBT_MALE_STGP_EVAL_N,DEBT_MALE_STGP_EVAL_MEAN,DEBT_MALE_STGP_EVAL_MDN,DEBT_MALE_PP_ANY_N,DEBT_MALE_PP_ANY_MEAN,DEBT_MALE_PP_ANY_MDN,DEBT_MALE_PP_EVAL_N,DEBT_MALE_PP_EVAL_MEAN,DEBT_MALE_PP_EVAL_MDN,DEBT_NOTMALE_STGP_ANY_N,DEBT_NOTMALE_STGP_ANY_MEAN,DEBT_NOTMALE_STGP_ANY_MDN,DEBT_NOTMALE_STGP_EVAL_N,DEBT_NOTMALE_STGP_EVAL_MEAN,DEBT_NOTMALE_STGP_EVAL_MDN,DEBT_NOTMALE_PP_ANY_N,DEBT_NOTMALE_PP_ANY_MEAN,DEBT_NOTMALE_PP_ANY_MDN,DEBT_NOTMALE_PP_EVAL_N,DEBT_NOTMALE_PP_EVAL_MEAN,DEBT_NOTMALE_PP_EVAL_MDN,DEBT_PELL_STGP_ANY_N,DEBT_PELL_STGP_ANY_MEAN,DEBT_PELL_STGP_ANY_MDN,DEBT_PELL_STGP_EVAL_N,DEBT_PELL_STGP_EVAL_MEAN,DEBT_PELL_STGP_EVAL_MDN,DEBT_PELL_PP_ANY_N,DEBT_PELL_PP_ANY_MEAN,DEBT_PELL_PP_ANY_MDN,DEBT_PELL_PP_EVAL_N,DEBT_PELL_PP_EVAL_MEAN,DEBT_PELL_PP_EVAL_MDN,DEBT_NOPELL_STGP_ANY_N,DEBT_NOPELL_STGP_ANY_MEAN,DEBT_NOPELL_STGP_ANY_MDN,DEBT_NOPELL_STGP_EVAL_N,DEBT_NOPELL_STGP_EVAL_MEAN,DEBT_NOPELL_STGP_EVAL_MDN,DEBT_NOPELL_PP_ANY_N,DEBT_NOPELL_PP_ANY_MEAN,DEBT_NOPELL_PP_ANY_MDN,DEBT_NOPELL_PP_EVAL_N,DEBT_NOPELL_PP_EVAL_MEAN,DEBT_NOPELL_PP_EVAL_MDN,DEBT_ALL_PP_ANY_MDN10YRPAY,DEBT_ALL_PP_EVAL_MDN10YRPAY,DEBT_ALL_STGP_ANY_MDN10YRPAY,DEBT_ALL_STGP_EVAL_MDN10YRPAY,EARN_COUNT_NWNE_HI_1YR,EARN_CNTOVER150_HI_1YR,EARN_COUNT_WNE_HI_1YR,EARN_MDN_HI_1YR,EARN_COUNT_NWNE_HI_2YR,EARN_CNTOVER150_HI_2YR,EARN_COUNT_WNE_HI_2YR,EARN_MDN_HI_2YR
0,100654.0,1002,Alabama A & M University,Public,1,100,"Agriculture, General.",3,Bachelors Degree,4.0,NaN,PrivacySuppressed,PrivacySuppressed,PrivacySuppressed,PrivacySuppressed,PrivacySuppressed,PrivacySuppressed,PrivacySuppressed,PrivacySuppressed,PrivacySuppressed,PrivacySuppressed,PrivacySuppressed,PrivacySuppressed,PrivacySuppressed,PrivacySuppressed,PrivacySuppressed,PrivacySuppressed,PrivacySuppressed,PrivacySuppressed,PrivacySuppressed,PrivacySuppressed,PrivacySuppressed,PrivacySuppressed,PrivacySuppressed,PrivacySuppressed,PrivacySuppressed,PrivacySuppressed,PrivacySuppressed,PrivacySuppressed,PrivacySuppressed,PrivacySuppressed,PrivacySuppressed,PrivacySuppressed,PrivacySuppressed,PrivacySuppressed,PrivacySuppressed,PrivacySuppressed,PrivacySuppressed,PrivacySuppressed,PrivacySuppressed,PrivacySuppressed,PrivacySuppressed,PrivacySuppressed,PrivacySuppressed,PrivacySuppressed,PrivacySuppressed,PrivacySuppressed,PrivacySuppressed,PrivacySuppressed,PrivacySuppressed,PrivacySuppressed,PrivacySuppressed,PrivacySuppressed,PrivacySuppressed,PrivacySuppressed,PrivacySuppressed,PrivacySuppressed,PrivacySuppressed,PrivacySuppressed,PrivacySuppressed,PrivacySuppressed,PrivacySuppressed,PrivacySuppressed,PrivacySuppressed,PrivacySuppressed,PrivacySuppressed,NaN,PrivacySuppressed,PrivacySuppressed,NaN,NaN,NaN,NaN
1,100654.0,1002,Alabama A & M University,Public,1,101,Agricultural Business and Management.,3,Bachelors Degree,NaN,NaN,PrivacySuppressed,PrivacySuppressed,PrivacySuppressed,PrivacySuppressed,PrivacySuppressed,PrivacySuppressed,PrivacySuppressed,PrivacySuppressed,PrivacySuppressed,PrivacySuppressed,PrivacySuppressed,PrivacySuppressed,PrivacySuppressed,PrivacySuppressed,PrivacySuppressed,PrivacySuppressed,PrivacySuppressed,PrivacySuppressed,PrivacySuppressed,PrivacySuppressed,PrivacySuppressed,PrivacySuppressed,PrivacySuppressed,PrivacySuppressed,PrivacySuppressed,PrivacySuppressed,PrivacySuppressed,PrivacySuppressed,PrivacySuppressed,PrivacySuppressed,PrivacySuppressed,PrivacySuppressed,PrivacySuppressed,PrivacySuppressed,PrivacySuppressed,PrivacySuppressed,PrivacySuppressed,PrivacySuppressed,PrivacySuppressed,PrivacySuppressed,PrivacySuppressed,PrivacySuppressed,PrivacySuppressed,PrivacySuppressed,Privac

According to the data dictionray, DEBT_ALL_STGP_EVAL_MDN is the Median Stafford and Grad PLUS loan debt disbursed at this institution

There are 218,901 records in the dataset

In [5]:
pysqldf("SELECT COUNT(*) FROM df")

,COUNT(*)
0,218901


Unfortunately, many have no data (small cohorts do not provide data due to privacy concerns)

In [6]:
df_id = pysqldf("""
SELECT 
    INSTNM, CIPDESC, EARN_MDN_HI_1YR, DEBT_ALL_STGP_EVAL_MDN 
FROM 
    df 
LIMIT 5
""")

I'm tracking 1yr median earnings, though you could modify this to track 2yr earnings if you'd like to stay consistent with the WSJ article. Keep in mind, some programs only report 1st year earnings, so you may be working with less data. It doesn't change much, though, 2nd year earnings provides a similar picture overall, though you may see some shuffling of programs/institutions or slightly different programs on the list. 

In [7]:
df_1yr = pysqldf("""
SELECT 
    INSTNM, 
    CIPCODE,
    CIPDESC, 
    CREDLEV, 
    CREDDESC, 
    EARN_MDN_HI_1YR * 1.0 AS EARN_MED, 
    DEBT_ALL_STGP_EVAL_MDN * 1.0 AS DEBT_MED
FROM 
    df 
WHERE 
    DEBT_ALL_STGP_EVAL_MDN != 'PrivacySuppressed'
    AND EARN_MDN_HI_1YR != 'PrivacySuppressed'
""")

Very heavy debt is typically for professional doctoral degrees in medicine or dentistry. Unfavorable ratios for medicine in particular may result residency programs which delay higher earnings. Medicine is probably better treated as a one off for 1st or 2nd year earnings, as salaries increase dramatically 5+ years after completion of residency. 

In [8]:
pysqldf("SELECT * FROM df_1yr ORDER BY DEBT_MED DESC LIMIT 20")

,INSTNM,CIPCODE,CIPDESC,CREDLEV,CREDDESC,EARN_MED,DEBT_MED
0,Roseman University of Health Sciences,5104,Dentistry.,6,Doctoral Degree,98400.0,410103.0
1,University of Southern California,5104,Dentistry.,7,First Professional Degree,85400.0,406131.0
2,Midwestern University-Downers Grove,5104,Dentistry.,6,Doctoral Degree,115700.0,396061.0
3,Western University of Health Sciences,5104,Dentistry.,7,First Professional Degree,106900.0,393548.0
4,New York University,5104,Dentistry.,7,First Professional Degree,69600.0,388447.0
5,Nova Southeastern University,5104,Dentistry.,6,Doctoral Degree,95600.0,379632.0
6,A T Still University of Health Sciences,5104,Dentistry.,7,First Professional Degree,101400.0,374924.0
7,University of the Pacific,5104,Dentistry.,7,First Professional Degree,87600.0,358760.0
8,"St. George's University, School of Medicine",5112,Medicine.,7,First Professional Degree,55200.0,351200.0
9,American University of the Caribbean,5112,Medicine.,7,First Professional Degree,52300.0,347604.0


#### Masters Degrees

Let's look specifically at masters degrees

In [9]:
pysqldf("""
SELECT 
    * 
FROM 
    df_1yr 
WHERE
    CREDLEV = 5
ORDER BY DEBT_MED DESC LIMIT 20
""")

,INSTNM,CIPCODE,CIPDESC,CREDLEV,CREDDESC,EARN_MED,DEBT_MED
0,Roseman University of Health Sciences,5202,"Business Administration, Management and Operations.",5,Master's Degree,115400.0,221021.0
1,Bastyr University,5133,Alternative and Complementary Medicine and Medical Systems.,5,Master's Degree,26200.0,202231.0
2,Lake Erie College of Osteopathic Medicine,1399,"Education, Other.",5,Master's Degree,161500.0,199840.0
3,Ohio State University-Main Campus,5104,Dentistry.,5,Master's Degree,231200.0,199145.0
4,University of Southern California,5109,"Allied Health Diagnostic, Intervention, and Treatment Professions.",5,Master's Degree,107700.0,198810.0
5,Logan University,5131,Dietetics and Clinical Nutrition Services.,5,Master's Degree,25400.0,183549.0
6,Pratt Institute-Main,402,Architecture.,5,Master's Degree,51000.0,181500.0
7,New York University,5006,Film/Video and Photographic Arts.,5,Master's Degree,29600.0,179680.0
8,Nova Southeastern University,5108,Allied Health and Medical Assisting Services.,5,Master's Degree,159200.0,177249.0
9,Columbia University in the City of New York,5006,Film/Video and Photographic Arts.,5,Master's Degree,25400.0,175429.0


Still mainly seeing health related MS degrees. Some of these degrees may be certifications earned after a professional degree, so 1st year may earnings reflect mid career earnings in a well renumerated field. 

We do see the MS in film at Columbia and NYT in there, though.

Instead of absolute debt, let's take a look at debt to income ratios for masters degrees

In [10]:
pysqldf("""
SELECT 
    INSTNM, 
    CIPDESC,
    EARN_MED,
    DEBT_MED,
    (DEBT_MED * 1.0) / (EARN_MED * 1.0) AS DEBT_RATIO
FROM 
    df_1yr 
WHERE
    CREDLEV = 5
ORDER BY DEBT_RATIO DESC LIMIT 100
""")

,INSTNM,CIPDESC,EARN_MED,DEBT_MED,DEBT_RATIO
0,Stanbridge University,Rehabilitation and Therapeutic Professions.,5500.0,83991.0,15.271091
1,Acupuncture and Integrative Medicine College-Berkeley,Alternative and Complementary Medicine and Medical Systems.,15200.0,120750.0,7.944079
2,Five Branches University,Alternative and Complementary Medicine and Medical Systems.,18600.0,147310.0,7.919892
3,Bastyr University,Alternative and Complementary Medicine and Medical Systems.,26200.0,202231.0,7.718740
4,New York Academy of Art,Fine and Studio Arts.,16000.0,121138.0,7.571125
5,Emperor's College of Traditional Oriental Medicine,Alternative and Complementary Medicine and Medical Systems.,15900.0,118659.0,7.462830
6,Logan University,Dietetics and Clinical Nutrition Services.,25400.0,183549.0,7.226339
7,Oregon College of Oriental Medicine,Alternative and Complementary Medicine and Medical Systems.,22100.0,159071.0,7.197783
8,Columbia University in the City of New York,Film/Video and Photographic Arts.,25400.0,175429.0,6.906654
9,New York University,Film/Video and Photographic Arts.,29600.0,179680.0,6.070270


Overall, the picture here is very consistent with the WSJ article. Film, Arts, Architecture, and Alternative Medicine Masters degree programs predominate among masters level programs with very high debt and low early career earnings. Social work, Psychology, and some natural science programs are also present.

#### Bachelors Degrees

The wsj article mentions that federal loans plaace much stricter upper limits on undergraduate borrowing than graduaate level borrowing. Let's take a look at the ratios at the bachelors degree level. The SQL query is almost identical to the one above, the only change is that we now use CREDLEV = 3.  

In [11]:
pysqldf("""
SELECT 
    INSTNM, 
    CIPDESC,
    EARN_MED,
    DEBT_MED,
    (DEBT_MED * 1.0) / (EARN_MED * 1.0) AS DEBT_RATIO
FROM 
    df_1yr 
WHERE
    CREDLEV = 3
ORDER BY DEBT_RATIO DESC LIMIT 100
""")

,INSTNM,CIPDESC,EARN_MED,DEBT_MED,DEBT_RATIO
0,Livingstone College,Social Work.,14500.0,45000.0,3.103448
1,Mississippi Valley State University,Social Work.,11600.0,35468.0,3.057586
2,AI Miami International University of Art and Design,Audiovisual Communications Technologies/Technicians.,15800.0,43194.0,2.733797
3,American InterContinental University-Atlanta,Film/Video and Photographic Arts.,18400.0,49579.0,2.694511
4,American InterContinental University,Film/Video and Photographic Arts.,18400.0,49579.0,2.694511
5,School of Visual Arts,Graphic Communications.,10200.0,27000.0,2.647059
6,Pontifical Catholic University of Puerto Rico-Ponce,"Registered Nursing, Nursing Administration, Nursing Research and Clinical Nursing.",7400.0,19250.0,2.601351
7,Memphis College of Art,Fine and Studio Arts.,17300.0,42750.0,2.471098
8,Southwest University at El Paso,Public Health.,20300.0,50031.0,2.464581
9,Grambling State University,Communication and Media Studies.,17400.0,42000.0,2.413793


The ratios here can be unfavorable, though nowhere near as severe as the highest debt to income ratio at the grad degree level. We are seeing a high concentration of art and social work. 

Let's look at the same query ordered by absolute debt. 

In [12]:
pysqldf("""
SELECT 
    INSTNM, 
    CIPDESC,
    EARN_MED,
    DEBT_MED,
    (DEBT_MED * 1.0) / (EARN_MED * 1.0) AS DEBT_RATIO
FROM 
    df_1yr 
WHERE
    CREDLEV = 3
ORDER BY DEBT_MED DESC LIMIT 100
""")

,INSTNM,CIPDESC,EARN_MED,DEBT_MED,DEBT_RATIO
0,Westwood College - South Bay,Criminal Justice and Corrections.,28100.0,57500.0,2.046263
1,ITT Technical Institute-Indianapolis,Construction Management.,61200.0,56394.0,0.921471
2,ITT Technical Institute-Indianapolis,Criminal Justice and Corrections.,29400.0,55309.0,1.881259
3,Westwood College - Denver North,Criminal Justice and Corrections.,26200.0,54529.0,2.081260
4,Sanford-Brown College-Tampa,Audiovisual Communications Technologies/Technicians.,26100.0,53946.0,2.066897
5,Sanford-Brown College-Las Vegas,Audiovisual Communications Technologies/Technicians.,26100.0,53946.0,2.066897
6,DeVry University-Arizona,Computer Engineering.,37800.0,53574.0,1.417302
7,DeVry University-California,Computer Engineering.,37800.0,53574.0,1.417302
8,DeVry University-Colorado,Computer Engineering.,37800.0,53574.0,1.417302
9,DeVry University-Florida,Computer Engineering.,37800.0,53574.0,1.417302


Interesting - computer engineering or information science from for-profit colleges are showing up a lot here. 

As a last query, outside the range of the article but still potentially interesting, let's look at debt for top paying Masters and Bachelors level programs. 

In [13]:
pysqldf("""
SELECT 
    INSTNM, 
    CIPDESC,
    EARN_MED,
    DEBT_MED,
    (DEBT_MED * 1.0) / (EARN_MED * 1.0) AS DEBT_RATIO
FROM 
    df_1yr 
WHERE
    CREDLEV = 5
ORDER BY EARN_MED DESC LIMIT 100
""")

,INSTNM,CIPDESC,EARN_MED,DEBT_MED,DEBT_RATIO
0,Ohio State University-Main Campus,Dentistry.,231200.0,199145.0,0.861354
1,University of Michigan-Ann Arbor,Advanced/Graduate Dentistry and Oral Sciences.,204000.0,170033.0,0.833495
2,University of Washington-Seattle Campus,Advanced/Graduate Dentistry and Oral Sciences.,197000.0,138818.0,0.704660
3,Central Connecticut State University,"Registered Nursing, Nursing Administration, Nursing Research and Clinical Nursing.",188500.0,44035.0,0.233607
4,Saint Mary's University of Minnesota,"Registered Nursing, Nursing Administration, Nursing Research and Clinical Nursing.",186000.0,60500.0,0.325269
5,University of New England,"Registered Nursing, Nursing Administration, Nursing Research and Clinical Nursing.",184700.0,132170.0,0.715593
6,University of Pennsylvania,Finance and Financial Management Services.,183600.0,41000.0,0.223312
7,University of Pennsylvania,International/Global Studies.,181700.0,103080.0,0.567309
8,Northwestern University,"Business Administration, Management and Operations.",181600.0,126678.0,0.697566
9,University of Pennsylvania,Management Sciences and Quantitative Methods.,181500.0,62311.0,0.343311


Looks like a lot of dentistry, nursing, and health sciences at the top, followed by business and some branches of computer science and engineering. Many of these programs have very high debt, though they also produce very high earnings. 

Engineering and CS masters degrees aren't quite as numerous in the top 100, though they tend to have lower debt ratios. A masters degree in computer science or engineering from a highly ranked public university does appear to be a pretty favorable option, at least from an income and debt perspective. Though now that I look at it... private colleges seem to offer a CS masters for considerably less than other degrees. Stanford has very low debt, and while Columbia CS masters may not produce quite as high a salary with such low debt as UC San Diego or Berkeley, it still looks like a favorable ROI. 

I wonder why a masters in CS from Columbia is less than 1/3 the cost of a masters in film? Well, according the the WSJ article, Columbia is using some degree programs to subsidize others. Another possibility is that MS students in engineering and CS are valuable as research and teaching associates in areas where it's hard to recruit faculty, so they get better financial aid packages and grant options. 

## Follow up article on Debt and Law Schools

The wsj referenced this data set in another article on graduate debt, this time in law schools. Law school students do often graduate with very heaavy debt. This isn't necessarily a problem from very elite schools where salaries are high. However, interestingly, tuition doesn't change much between elite and mid-tier (or even lower-tier) law schools. 

https://www.wsj.com/articles/law-school-student-debt-low-salaries-university-miami-11627991855

Let's look at the claim "Only a dozen of the nation’s law schools leave students earning annual salaries two years after graduation that exceed their debts".

The wsj article mentions 2nd year salary, but I'm not seeing any data for this, only 1yr. Maybe I'm using a different dataset? I'll try the calculations with first year out salary, and see how close I can get.

Note - because a JD is sometimes listed as a first professional degree and sometimes as a doctoral degree, I'll  use the CIPCODE for law schools. 

In [43]:
df_law = pysqldf("""
SELECT 
    INSTNM, 
    CIPCODE,
    CIPDESC, 
    CREDLEV, 
    CREDDESC, 
    EARN_MDN_HI_1YR * 1.0 AS EARN_MED_1YR, 
    EARN_MDN_HI_2YR * 1.0 AS EARN_MED_2YR, 
    DEBT_ALL_STGP_EVAL_MDN * 1.0 AS DEBT_MED
FROM 
    df 
WHERE
    CIPCODE == 2201
--WHERE 
--    DEBT_ALL_STGP_EVAL_MDN != 'PrivacySuppressed'
--    AND EARN_MDN_HI_2YR != 'PrivacySuppressed'
""")

In [50]:
df_law.head()

,INSTNM,CIPCODE,CIPDESC,CREDLEV,CREDDESC,EARN_MED_1YR,EARN_MED_2YR,DEBT_MED
0,The University of Alabama,2201,Law.,7,First Professional Degree,51900.0,None,68485.0
1,Faulkner University,2201,Law.,7,First Professional Degree,36300.0,None,0.0
2,Samford University,2201,Law.,6,Doctoral Degree,44800.0,None,135438.0
3,Samford University,2201,Law.,7,First Professional Degree,0.0,None,0.0
4,Arizona State University-Tempe,2201,Law.,6,Doctoral Degree,56800.0,None,94325.0


Unfortunately, we have no data for 1st year median earnings, and some data is reported as 0. We'll have to drop those rows in the query, even if it means losing some data.

In [53]:
df_law.count()

INSTNM          440
CIPCODE         440
CIPDESC         440
CREDLEV         440
CREDDESC        440
EARN_MED_1YR    440
EARN_MED_2YR      0
DEBT_MED        440
dtype: int64

Looks like we have complete data for 1st year, but no 2nd year earnings. 

So let's try the calcs for 1st year. 

In [55]:
pysqldf("""
SELECT 
    INSTNM, 
    CIPDESC,
    CREDLEV,
    EARN_MED_1YR,
    DEBT_MED,
    (DEBT_MED * 1.0) / (EARN_MED_1YR * 1.0) AS DEBT_RATIO
FROM 
    df_law 
WHERE
    CIPCODE == 2201
AND
    EARN_MED_1YR * 1.0 > 0
AND
    DEBT_MED * 1.0 > 0
ORDER BY DEBT_RATIO ASC LIMIT 100
""")

,INSTNM,CIPDESC,CREDLEV,EARN_MED_1YR,DEBT_MED,DEBT_RATIO
0,Stanford University,Law.,7,156700.0,120422.0,0.768488
1,Harvard University,Law.,7,158200.0,133617.0,0.844608
2,Duke University,Law.,6,162200.0,138000.0,0.850801
3,University of Pennsylvania,Law.,7,175100.0,149729.0,0.855106
4,University of Chicago,Law.,7,170500.0,146806.0,0.861032
5,Cornell University,Law.,6,175200.0,153937.0,0.878636
6,Northwestern University,Law.,7,171900.0,156110.0,0.908144
7,Columbia University in the City of New York,Law.,7,180300.0,165500.0,0.917915
8,Brigham Young University-Provo,Law.,6,56200.0,52520.0,0.934520
9,Yale University,Law.,6,128900.0,126398.0,0.980590


So I'm getting numbers very similar to the WSJ article. For my query, I'm seeing 10 schools where salaries are higher than debt, but that's not a big difference, and I am using 1st year rather than 2nd year salaries (not sure where they got that data)

One interesting thing here is that unlike engineering, there's a world of difference between attending a top 10 law and top 100 engineering graduate program. Here's a query for a group of masters in computer science or masters degrees with the words "engineering" or "computer science in them (I'm too lazy to to track all the CIPCODEs for all engineering programs, though I can use the c). 

In [66]:
pysqldf("""
SELECT 
    CIPCODE,
    INSTNM, 
    CIPDESC,
    CREDLEV,
    EARN_MDN_HI_1YR * 1.0 AS EARN_MED_1YR,
    DEBT_ALL_STGP_EVAL_MDN * 1.0 AS DEBT_MED,
    (DEBT_ALL_STGP_EVAL_MDN * 1.0) / (EARN_MDN_HI_1YR * 1.0) AS DEBT_RATIO
FROM 
    df 
WHERE
    (CIPDESC LIKE '%Engineering%' OR CIPDESC LIKE '%Computer Science%' )
AND 
    CREDLEV = 5
AND
    EARN_MED_1YR * 1.0 > 0
AND
    DEBT_MED * 1.0 > 0
ORDER BY DEBT_RATIO ASC LIMIT 100
""")

,CIPCODE,INSTNM,CIPDESC,CREDLEV,EARN_MED_1YR,DEBT_MED,DEBT_RATIO
0,1427,Johns Hopkins University,Systems Engineering.,5,109900.0,15382.0,0.139964
1,1107,University of California-San Diego,Computer Science.,5,141200.0,24669.0,0.174710
2,1410,Binghamton University,"Electrical, Electronics and Communications Engineering.",5,79400.0,14000.0,0.176322
3,1107,Stanford University,Computer Science.,5,153400.0,27400.0,0.178618
4,1410,University of California-Los Angeles,"Electrical, Electronics and Communications Engineering.",5,90700.0,16620.0,0.183241
5,1410,Georgia Institute of Technology-Main Campus,"Electrical, Electronics and Communications Engineering.",5,96400.0,17932.0,0.186017
6,1410,Iowa State University,"Electrical, Electronics and Communications Engineering.",5,92000.0,18000.0,0.195652
7,1401,Old Dominion University,"Engineering, General.",5,73500.0,15115.0,0.205646
8,1410,California Polytechnic State University-San Luis Obispo,"Electrical, Electronics and Communications Engineering.",5,97200.0,19995.0,0.205710
9,1419,Stony Brook University,Mechanical Engineering.,5,67600.0,14000.0,0.207101


Debt doesn't look like a big problem for top 100 MS programs in CS or Engineering, even aat the lower earnings level. Since debt isn't really a problem, let's look at this ranked by top salary rather than indebtedness.

In [68]:
pysqldf("""
SELECT 
    CIPCODE,
    INSTNM, 
    CIPDESC,
    CREDLEV,
    EARN_MDN_HI_1YR * 1.0 AS EARN_MED_1YR,
    DEBT_ALL_STGP_EVAL_MDN * 1.0 AS DEBT_MED,
    (DEBT_ALL_STGP_EVAL_MDN * 1.0) / (EARN_MDN_HI_1YR * 1.0) AS DEBT_RATIO
FROM 
    df 
WHERE
    (CIPDESC LIKE '%Engineering%' OR CIPDESC LIKE '%Computer Science%' )
AND 
    CREDLEV = 5
AND
    EARN_MED_1YR * 1.0 > 0
AND
    DEBT_MED * 1.0 > 0
ORDER BY EARN_MED_1YR DESC LIMIT 100
""")

,CIPCODE,INSTNM,CIPDESC,CREDLEV,EARN_MED_1YR,DEBT_MED,DEBT_RATIO
0,1107,Stanford University,Computer Science.,5,153400.0,27400.0,0.178618
1,1107,University of California-San Diego,Computer Science.,5,141200.0,24669.0,0.174710
2,1107,Columbia University in the City of New York,Computer Science.,5,132500.0,46100.0,0.347925
3,1499,Northwestern University,"Engineering, Other.",5,131900.0,100788.0,0.764124
4,1427,Massachusetts Institute of Technology,Systems Engineering.,5,130600.0,41000.0,0.313936
5,1410,Stanford University,"Electrical, Electronics and Communications Engineering.",5,126700.0,30271.0,0.238919
6,1107,Cornell University,Computer Science.,5,113700.0,30704.0,0.270044
7,1410,Carnegie Mellon University,"Electrical, Electronics and Communications Engineering.",5,113000.0,30353.0,0.268611
8,1410,The University of Texas at Austin,"Electrical, Electronics and Communications Engineering.",5,110400.0,26302.0,0.238243
9,1427,Johns Hopkins University,Systems Engineering.,5,109900.0,15382.0,0.139964
